In [1]:
#===========================================================================================================
#                                    Librerías necesarias para correr el código
#===========================================================================================================
# Librerías para hacer llamadas con el Servicio Web y rescatar la información en formato JSON
import urllib, json
# Librería para crear/manipular la base de datos
import pandas as pd
# Librería para manipulación de archivos
import os.path
# Librería para manipulación de fechas
from datetime import date
from datetime import timedelta

In [2]:
#===========================================================================================================
#                                        Definición de Variables Utilizadas
#===========================================================================================================
# Se definen las zonas que se analizan/añaden a la base de datos. Estas zonas son de la región Occidental
Zonas = ["AGUASCALIENTES", "APATZINGAN", "CELAYA", "CIENEGA", "COLIMA", "FRESNILLO", "GUADALAJARA", "IRAPUATO",
           "IXMIQUILPAN", "JIQUILPAN", "LEON", "LOS-ALTOS", "MANZANILLO", "MATEHUALA", "MINAS", "MORELIA", "QUERETARO",
           "SALVATIERRA", "SAN-JUAN-DEL-RIO", "SAN-LUIS-POTOSI", "TEPIC-VALLARTA", "URUAPAN", "ZACAPU", "ZACATECAS",
           "ZAMORA", "ZAPOTLAN"]

# Se definen los nombres de las columnas recogidas de la llamada URL en formato JSON del Servicio Web de CENACE
Columnas = ["fecha", "hora", "pz", "pz_cng", "pz_ene", "pz_per", "zona_carga", "Dia_de_la_semana", "Festivo"]

In [3]:
#===========================================================================================================
#                                        Definición de Funciones Utilizadas
#===========================================================================================================
# Se define una función que agrega 0's para que los días tengan formato "DD"
def Dia(fecha):
    """Esta función se encarga de agregar un 0 a los primeros 9 días del mes introduciendo como variable un elemento que
    se encuentre en formato de fecha (perteneciente a la librería "datetime")."""
    if fecha.day < 10:
        return "0" + str(fecha.day)
    else:
        return str(fecha.day)

# Similar a la función "Día", esta función se encarga de agregar 0's para que los meses tengan formato "MM"   
def Mes(fecha):
    """Esta función se encarga de agregar un 0 a los primeros 9 meses del año introduciendo como variable un elemento
    que se encuentre en formato de fecha (perteneciente a la librería "datetime")."""
    if fecha.month < 10:
        return "0" + str(fecha.month)
    else:
        return str(fecha.month)

# Función generadora de enlaces URL's para el Servicio Web de CENACE
def ruta_descarga(zona_sel, Inicial, proceso):
    """Esta función se encarga de generar la ruta de descarga a partir del formato del Servicio Web de CENACE. Para
    hacer que dicha función trabaje, es necesario introducir una zona o selección de zonas (formato de arreglo) y la
    fecha inicial a partir de la cual se quieren comenzar a recoger los datos."""
    # Ruta base para la descarga
    ruta = "https://ws01.cenace.gob.mx:8082/SWPEND/SIM/"
    # Sistema Interconectado: Sistema Interconectado Nacional (SIN), Sistema Interconectado Baja Californa (BCA) y
    # Sistema Interconectado Baja California Sur (BCS)
    sis_int = "SIN"
    # Proceso que se va a utilizar: MDA o MTR
    proc = proceso
    # Se juntan las variables anteriores en una sola ruta
    ruta = ruta + sis_int + "/" + proc + "/"
    # Se agrega la zona (o las zonas) a la ruta del URL. En caso de tener múltiples zonas, se agregan comas para separar
    # los lugares que se desean analizar
    for zona in (zona_sel):
        ruta = ruta + zona + ","
    # Con la variable de fecha que se introduce (desde qué fecha se desea descargar), se agregan los datos al URL
    ruta = ruta[:-1] + "/" + str(Inicial.year) + "/" + Mes(Inicial) + "/" + Dia(Inicial) + "/"
    # Se define la fecha final con un desplazamiento de 6 días (tiempo límite de consulta con el Servicio Web = 7 días)
    Final = Inicial + timedelta(days = 6)
    # Se añaden los nuevos datos al URL
    ruta = ruta + str(Final.year) + "/" + Mes(Final) + "/" + Dia(Final) + "/"
    # Formato de salida: XML o JSON (en minúsculas)
    formato = "json"
    # Se crea el URL final de acceso al Servicio Web
    ruta = ruta + formato
    print(ruta)
    return ruta

# Función que permite extraer la información de la llamada al Servicio Web en formato JSON
def getDF(ruta):
    """Esta función se encarga de acceder a la información de Internet y extraer los datos."""
    s = "/"
    # Se genera una variable que almacena los datos de la llamada al URL
    data = json.loads(urllib.request.urlopen(s.join(ruta)).read())
    # El JSON se utiliza como diccionario y se obtiene el "key" que contiene la información necesaria.
    resultados = {key: value for key, value in data.items() if key == "Resultados"}.get("Resultados")
    # Se hace de manera iterativa en caso de que sean más de una zona de carga y se agrega a un dataframe
    df = pd.concat([pd.DataFrame(pd.DataFrame(i)["Valores"].tolist()).join(pd.DataFrame(i)["zona_carga"]) for i in resultados])
    return df

# Esta función se encarga de renombrar las columnas originales de la llamada en JSON en las columnas finales de la base
# de datos
def Renombrar(archivo_csv):
    """Esta función se encarga de renombrar las columnas originales de la llamada en JSON."""
    archivo_csv.columns = [column.replace("pz_ene","Componente_Energia") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("pz_cng","Componente_Congestion") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("pz_per","Componente_Perdidas") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("zona_carga","Zona_de_Carga") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("pz","Precio_Zonal") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("fecha","Fecha") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("hora","Hora") for column in archivo_csv.columns]

# Esta función se encarga de remplazar los nombres de las columnas de la base de datos final a las columnas iniciales
# de la llamada en JSON
def RenombrarInv(archivo_csv):
    """Esta función es inversa a la función 'Renombrar' y se encarga de regresar los nombres de las columnas originales.
    Este cambio es necesario cuando se requiere añadir información nueva a la base de datos existente."""
    archivo_csv.columns = [column.replace("Precio_Zonal","pz") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("Componente_Congestion","pz_cng") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("Componente_Energia","pz_ene") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("Componente_Perdidas","pz_per") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("Zona_de_Carga","zona_carga") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("Fecha","fecha") for column in archivo_csv.columns]
    archivo_csv.columns = [column.replace("Hora",'hora') for column in archivo_csv.columns]

# Esta función se encarga de añadir el día de la semana correspondiente a la fecha en la base de datos
def Dia_Semana(archivo_csv):
    """Esta función se encarga de añadir el día de la semana correspondiente (Lunes, Martes, Miércoles, etc.)."""
    days = {0:"Lunes", 1:"Martes", 2:"Miercoles", 3: "Jueves", 4:"Viernes", 5:"Sabado", 6:"Domingo"}
    archivo_csv["Fecha"] = pd.to_datetime(archivo_csv["Fecha"])
    archivo_csv["Dia_de_la_semana"] = archivo_csv["Fecha"].dt.dayofweek
    archivo_csv["Dia_de_la_semana"] = archivo_csv["Dia_de_la_semana"].apply(lambda x: days[x])
    
# Esta función se encarga de informar si el día es festivo y a qué festividad corresponde
def Festivos(archivo_csv):
    """Esta función se encarga de agregar el día festivo, tanto en fecha como en qué día festivo."""
    ruta = r"Festivos.csv"
    Fest = pd.read_csv(ruta)
    Fest["Fecha"]= pd.to_datetime(Fest["Fecha"])
    Fest["Fecha"] = Fest["Fecha"].dt.strftime("%Y-%m-%d")
    Fest["Fecha"]= pd.to_datetime(Fest["Fecha"])
    archivo_csv = pd.merge(archivo_csv, Fest, on = ["Fecha", "Zona_de_Carga"], how = "left")
    return archivo_csv

In [4]:
#===========================================================================================================
#                                        Descarga y Optimización de Archivos MTR
#===========================================================================================================
# Se crea el data frame de pandas con las Columnas definidas en la sección de variables
BaseDatos = pd.DataFrame(columns = Columnas)
# Se almacena la fecha actual correspondiente al sistema y se agrega un día más (para lograr recoger los datos más
# nuevos de los precios de nodos en el proceso MTR)
Hoy = date.today() + timedelta(days = 1)

# Se revisa si ya existe una base de datos
if os.path.exists("BaseDatosMTR.csv"):
    # Si la base ya existe, se revisa la última fecha añadida y con ello se genera una nueva fecha para el ULR del
    # Servicio Web
    BaseDatos = pd.read_csv("BaseDatosMTR.csv")
    RenombrarInv(BaseDatos)
    BaseDatos = BaseDatos[Columnas]
    Ini = (BaseDatos["fecha"]).max()
    # El formato de fecha "AAAA-MM-DD" de la base de datos se transforma a formato de la librería "datetime" 
    Ini = date(int(Ini[0:4]), int(Ini[5:7]), int(Ini[8:10])) + timedelta(days = 1)
else:
    # Se crea la fecha del primer registro de CENACE con formato (año, mes, día) en caso de que no existan archivos
    # previos. Esto permite generar la base de datos desde la primera entrada
    
    # 27 de enero de 2017. MTR --> Ini = date(2017,1,27)
    # 29 de enero de 2016. MTR --> Ini = date(2016,1,29)
    Ini = date(2017,1,27)
# Se seleccionan 10 zonas para agregarlas al URL y posteriormente a la Base de Datos
for i in range(1 + len(Zonas)//10):
    zona_sel = Zonas[10 * i : 10 * i + 10]
    Inicial = Ini
    # Hasta que la fecha introducida inicialmente (ya sea desde 0 o desde una fecha previa) no sea igual a la del
    # sistema, el programa seguirá generando URLs y descargando la información para introducirla al data frame
    while Inicial <= Hoy:
        try:
            df = getDF([ruta_descarga(zona_sel,Inicial,"MTR")])
            Renombrar(df)
            Dia_Semana(df)
            df = Festivos(df)
            RenombrarInv(df)
            BaseDatos = BaseDatos.append(df[Columnas], ignore_index = True)
        except Exception as ex:
            break
        Inicial = Inicial + timedelta(days=7)

# Se renombran las columnas de la base de datos
Renombrar(BaseDatos)
# Se genera el archivo final en formato "csv"
BaseDatos.to_csv("BaseDatosMTR.csv", index = False)

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/01/27/2017/02/02/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/02/03/2017/02/09/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/02/10/2017/02/16/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/02/17/2017/02/23/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/02/24/2017/03/02/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNI

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/12/15/2017/12/21/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/12/22/2017/12/28/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/12/29/2018/01/04/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/01/05/2018/01/11/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/01/12/2018/01/18/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNI

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/11/02/2018/11/08/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/11/09/2018/11/15/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/11/16/2018/11/22/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/11/23/2018/11/29/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/11/30/2018/12/06/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNI

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2019/09/20/2019/09/26/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2019/09/27/2019/10/03/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2019/10/04/2019/10/10/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2019/10/11/2019/10/17/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2019/10/18/2019/10/24/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNI

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2020/08/07/2020/08/13/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2020/08/14/2020/08/20/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2020/08/21/2020/08/27/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2017/01/27/2017/02/02/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2017/02/03/2017/02/09/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORE

...

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2019/08/09/2019/08/15/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2019/08/16/2019/08/22/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2019/08/23/2019/08/29/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2019/08/30/2019/09/05/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2019/09/06/2019/09/12/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUAL

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2020/06/19/2020/06/25/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2020/06/26/2020/07/02/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2020/07/03/2020/07/09/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2020/07/10/2020/07/16/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2020/07/17/2020/07/23/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/LEON,LOS-ALTOS,MANZANILLO,MATEHUAL

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2017/12/29/2018/01/04/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2018/01/05/2018/01/11/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2018/01/12/2018/01/18/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2018/01/19/2018/01/25/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2018/01/26/2018/02/01/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2018/02/02/2018/02/08/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2018/02/09/2018/02/15/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLA

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/03/08/2019/03/14/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/03/15/2019/03/21/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/03/22/2019/03/28/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/03/29/2019/04/04/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/04/05/2019/04/11/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/04/12/2019/04/18/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/04/19/2019/04/25/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLA

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/05/15/2020/05/21/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/05/22/2020/05/28/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/05/29/2020/06/04/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/06/05/2020/06/11/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/06/12/2020/06/18/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/06/19/2020/06/25/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/06/26/2020/07/02/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MTR/TEPIC-VALLA

In [5]:
#===========================================================================================================
#                                        Descarga y Optimización de Archivos MDA
#===========================================================================================================
# Se crea el data frame de pandas con las Columnas definidas en la sección de variables
BaseDatos = pd.DataFrame(columns = Columnas)
# Se almacena la fecha actual correspondiente al sistema y se agrega un día más (para lograr recoger los datos más
# nuevos de los precios de nodos en el proceso MDA)
Hoy = date.today() + timedelta(days = 1)

# Se revisa si ya existe una base de datos
if os.path.exists("BaseDatosMDA.csv"):
    # Si la base ya existe, se revisa la última fecha añadida y con ello se genera una nueva fecha para el ULR del
    # Servicio Web
    BaseDatos = pd.read_csv("BaseDatosMDA.csv")
    RenombrarInv(BaseDatos)
    BaseDatos = BaseDatos[Columnas]
    Ini = (BaseDatos["fecha"]).max()
    # El formato de fecha "AAAA-MM-DD" de la base de datos se transforma a formato de la librería "datetime" 
    Ini = date(int(Ini[0:4]), int(Ini[5:7]), int(Ini[8:10])) + timedelta(days = 1)
else:
    # Se crea la fecha del primer registro de CENACE con formato (año, mes, día) en caso de que no existan archivos
    # previos. Esto permite generar la base de datos desde la primera entrada
    
    # 27 de enero de 2017. MDA --> Ini = date(2017,1,27)
    # 29 de enero de 2016. MDA --> Ini = date(2016,1,29)
    Ini = date(2017,1,27)
# Se seleccionan 10 zonas para agregarlas al URL y posteriormente a la Base de Datos
for i in range(1 + len(Zonas)//10):
    zona_sel = Zonas[10 * i : 10 * i + 10]
    Inicial = Ini
    # Hasta que la fecha introducida inicialmente (ya sea desde 0 o desde una fecha previa) no sea igual a la del
    # sistema, el programa seguirá generando URLs y descargando la información para introducirla al data frame
    while Inicial <= Hoy:
        try:
            df = getDF([ruta_descarga(zona_sel,Inicial,"MDA")])
            Renombrar(df)
            Dia_Semana(df)
            df = Festivos(df)
            RenombrarInv(df)
            BaseDatos = BaseDatos.append(df[Columnas], ignore_index = True)
        except Exception as ex:
            break
        Inicial = Inicial + timedelta(days=7)

# Se renombran las columnas de la base de datos
Renombrar(BaseDatos)
# Se genera el archivo final en formato "csv"
BaseDatos.to_csv("BaseDatosMDA.csv", index = False)

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/01/27/2017/02/02/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/02/03/2017/02/09/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/02/10/2017/02/16/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/02/17/2017/02/23/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/02/24/2017/03/02/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNI

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/12/15/2017/12/21/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/12/22/2017/12/28/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2017/12/29/2018/01/04/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/01/05/2018/01/11/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/01/12/2018/01/18/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNI

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/11/02/2018/11/08/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/11/09/2018/11/15/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/11/16/2018/11/22/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/11/23/2018/11/29/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2018/11/30/2018/12/06/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNI

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2019/09/20/2019/09/26/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2019/09/27/2019/10/03/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2019/10/04/2019/10/10/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2019/10/11/2019/10/17/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2019/10/18/2019/10/24/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNI

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2020/08/07/2020/08/13/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2020/08/14/2020/08/20/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2020/08/21/2020/08/27/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/AGUASCALIENTES,APATZINGAN,CELAYA,CIENEGA,COLIMA,FRESNILLO,GUADALAJARA,IRAPUATO,IXMIQUILPAN,JIQUILPAN/2020/08/28/2020/09/03/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2017/01/27/2017/02/02/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,

...

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2019/08/02/2019/08/08/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2019/08/09/2019/08/15/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2019/08/16/2019/08/22/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2019/08/23/2019/08/29/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2019/08/30/2019/09/05/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUAL

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2020/06/12/2020/06/18/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2020/06/19/2020/06/25/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2020/06/26/2020/07/02/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2020/07/03/2020/07/09/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUALA,MINAS,MORELIA,QUERETARO,SALVATIERRA,SAN-JUAN-DEL-RIO,SAN-LUIS-POTOSI/2020/07/10/2020/07/16/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/LEON,LOS-ALTOS,MANZANILLO,MATEHUAL

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2017/12/08/2017/12/14/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2017/12/15/2017/12/21/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2017/12/22/2017/12/28/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2017/12/29/2018/01/04/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2018/01/05/2018/01/11/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2018/01/12/2018/01/18/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2018/01/19/2018/01/25/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLA

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/02/15/2019/02/21/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/02/22/2019/02/28/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/03/01/2019/03/07/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/03/08/2019/03/14/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/03/15/2019/03/21/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/03/22/2019/03/28/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2019/03/29/2019/04/04/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLA

https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/04/24/2020/04/30/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/05/01/2020/05/07/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/05/08/2020/05/14/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/05/15/2020/05/21/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/05/22/2020/05/28/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/05/29/2020/06/04/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLARTA,URUAPAN,ZACAPU,ZACATECAS,ZAMORA,ZAPOTLAN/2020/06/05/2020/06/11/json
https://ws01.cenace.gob.mx:8082/SWPEND/SIM/SIN/MDA/TEPIC-VALLA

In [2]:
Data1 = pd.read_csv("BaseDatosMDA.csv")
Data1.head()

,Fecha,Hora,Precio_Zonal,Componente_Congestion,Componente_Energia,Componente_Perdidas,Zona_de_Carga,Dia_de_la_semana,Festivo
0,2017-01-27,1,707.96,18.94,685.03,3.99,AGUASCALIENTES,Viernes,NaN
1,2017-01-27,2,652.18,12.26,636.91,3.01,AGUASCALIENTES,Viernes,NaN
2,2017-01-27,3,626.64,9.97,612.97,3.70,AGUASCALIENTES,Viernes,NaN
3,2017-01-27,4,611.90,8.53,600.11,3.26,AGUASCALIENTES,Viernes,NaN
4,2017-01-27,5,588.25,5.45,578.50,4.31,AGUASCALIENTES,Viernes,NaN


In [3]:
len(Data1)

823294

In [4]:
Data2 = pd.read_csv("BaseDatosMTR.csv")
Data2.head()

C:\Users\OF\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3071: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,Fecha,Hora,Precio_Zonal,Componente_Congestion,Componente_Energia,Componente_Perdidas,Zona_de_Carga,Dia_de_la_semana,Festivo
0,2017-01-27,1,689.71,8.67,674.62,6.43,AGUASCALIENTES,Viernes,NaN
1,2017-01-27,2,633.81,2.45,625.16,6.20,AGUASCALIENTES,Viernes,NaN
2,2017-01-27,3,625.77,2.07,619.15,4.55,AGUASCALIENTES,Viernes,NaN
3,2017-01-27,4,605.57,0.00,599.94,5.64,AGUASCALIENTES,Viernes,NaN
4,2017-01-27,5,609.05,0.00,604.43,4.61,AGUASCALIENTES,Viernes,NaN


In [5]:
len(Data2)

817054

In [7]:
df_global = pd.concat([Data1, Data2])
df_global.to_csv("BaseDatos.csv")